SciPy 的 BFGS：

对于小型问题，收敛性较好，稳定性经过多年验证。

但在高维或病态问题（Hessian 矩阵条件数较高）上可能因内存限制或数值误差表现不佳。

JAX 的 L-BFGS：

L-BFGS 的有限内存特性使其在大规模问题上更稳定，尤其在高维空间中。

收敛速度可能略慢于完整 BFGS，但通过调整历史向量数量（
𝑚
m）可优化性能。

结合 JAX 的精确梯度和数值稳定性，适合复杂的非凸优化问题。

当不给定梯度计算的函数时，SCIPY的BFGS不会收敛到0，这是一个小缺点。但计算要比JAX快速。

In [20]:
from scipy.optimize import minimize
import numpy as np

def objective(x):
    return x[0]**2 + x[1]**2 

def gradient(x):
    return np.array([2*x[0], 2*x[1]])

result = minimize(objective, x0=np.array([1.0, 1.0]), method='BFGS')
print(result.x)


[-1.07505143e-08 -1.07505143e-08]


In [15]:
import jax
from jax import jit
import jax.numpy as jnp
from jaxopt import LBFGS

@jit
def objective(x):
    return x[0]**2 + x[1]**2

lbfgs = LBFGS(fun=objective, maxiter=100)
x0 = jnp.array([1.0, 1.0])
result = lbfgs.run(x0)
print(result.params)


[0. 0.]


In [21]:
import jax
import jax.numpy as jnp
from pso_jax import PSO  # PSO 是主要的优化器类

# 定义简单的适应度函数：在重量约束下最大化总价值
values = jnp.array([10, 15, 20, 25])
weights = jnp.array([2, 3, 5, 7])
capacity = 10

def fitness(bitmask):
    # bitmask ∈ [0,1]^4 连续值 → 后续四舍五入成 0/1
    selection = jnp.round(bitmask)
    total_weight = jnp.dot(selection, weights)
    total_value  = jnp.dot(selection, values)
    # 超重则惩罚为 0
    return jnp.where(total_weight <= capacity, total_value, 0.0)

# 初始化 PSO 优化器
pso = PSO(
    dim=values.shape[0],          # 问题维度
    num_particles=50,             # 粒子群规模
    inertia=0.7,                  # 惯性系数
    cognitive=1.5, social=1.5,    # 认知与社交加速系数
    bounds=(0.0, 1.0)             # 搜索空间为 [0,1]
)

# 随机种子
key = jax.random.PRNGKey(42)

# 运行优化：返回最优位置和最优适应度
best_pos, best_val = pso.optimize(key, fitness, num_iters=100)

print("最优二进制选择：", jnp.round(best_pos))
print("找到的最优价值：  ", best_val)


ModuleNotFoundError: No module named 'pso_jax'

In [22]:
! pip install git+https://github.com/BHI-Research/PSO-JAX.git

  Cloning https://github.com/BHI-Research/PSO-JAX.git to c:\users\19464\appdata\local\temp\pip-req-build-3yj62i5j
  Resolved https://github.com/BHI-Research/PSO-JAX.git to commit b73566784f138cf37db70aac97ef84ea8c70889a


  Running command git clone --filter=blob:none --quiet https://github.com/BHI-Research/PSO-JAX.git 'C:\Users\19464\AppData\Local\Temp\pip-req-build-3yj62i5j'
ERROR: git+https://github.com/BHI-Research/PSO-JAX.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
